In [37]:
import pandas as pd
import numpy as np
import plotly as py
from plotly import tools
import plotly.graph_objs as go
from feature_functions import *
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from scipy import stats
import scipy.optimize
from scipy.optimize import OptimizeWarning
import warnings
from sklearn.linear_model import LinearRegression

# Variables:
future = 15
averages = [5,10,15,20,30,50,70,100,200,300]


# Loading the data
df = pd.read_csv("data/EURUSDhour.csv")
df.columns = ['date','open','high','low','close','volume']
df.date = pd.to_datetime(df.date, format='%d.%m.%Y %H:%M:%S.%f')
df = df.set_index(df.date)
df = df[['open','high','low','close','volume']]

# Drop elements where there is no mouvement (Market closed) like in the weekends
df = df.drop_duplicates(keep=False)
df

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-37-9863bb697ac2>", line 23, in <module>
    df = pd.read_csv("data/EURUSDhour.csv")
  File "/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py", line 678, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py", line 440, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py", line 787, in __init__
    self._make_engine(self.engine)
  File "/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py", line 1014, in _make_engine
    self._engine = CParserWrapper(self.f, **self.options)
  File "/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py", line 1708, in __init__
    self._reader = parsers.TextReader(src, *

FileNotFoundError: File b'data/EURUSDhour.csv' does not exist

In [58]:
# Bollinger Bands

def bollinger(prices, periods, deviations):
    """

    :param prices: OHLC data
    :param periods: periods for which to compute the bollinger bands
    :param deviations: deviations to use when calculating bands(upper and lower)
    :return: bollinger bands
    """

    results = pd.DataFrame(index=prices.index)
    boll = {}

    for i in range(len(periods)):
        mid = prices.close.rolling(periods[i]).mean()
        std = prices.close.rolling(periods[i]).std()

        upper = mid + deviations * std
        lower = mid - deviations * std

        df = pd.concat((upper, mid, lower), axis=1)
        df.columns = ['upper bollinger' + str(periods[i]), 'mid bollinger' + str(periods[i]), 'lower bollinger' + str(periods[i])]

        results = pd.concat([results, df], axis=1)


    return results


In [59]:
bollingerKey = [15]

bollinger(df, bollingerKey, 2)

,upper bollinger15,mid bollinger15,lower bollinger15
date,,,
2017-01-01 22:00:00,NaN,NaN,NaN
2017-01-01 23:00:00,NaN,NaN,NaN
2017-01-02 00:00:00,NaN,NaN,NaN
2017-01-02 01:00:00,NaN,NaN,NaN
2017-01-02 02:00:00,NaN,NaN,NaN
2017-01-02 03:00:00,NaN,NaN,NaN
2017-01-02 04:00:00,NaN,NaN,NaN
2017-01-02 05:00:00,NaN,NaN,NaN
2017-01-02 06:00:00,NaN,NaN,NaN


In [ ]:
# The data
interval = 20
run_length = 100
correct_predictions = 0
for i in range(run_length):
    X = df_with_averages.iloc[:df_with_averages.shape[0]- (i*interval),:-1]
    y = df_with_averages.result

    # Split the data
    X_train= X.iloc[:X.shape[0]-1,:]
    X_test= X.iloc[X.shape[0]-1:X.shape[0],:]
    y_train= y[:X.shape[0]-1]
    y_test =y [X.shape[0]-1:X.shape[0]]

    clf = GradientBoostingClassifier(random_state=5, learning_rate=0.01, n_estimators=10000)
    clf.fit(X_train, y_train) 

    # Predictions
    predicted = clf.predict(X_test)
    print (str(accuracy_score(y_test, predicted)))
    correct_predictions += accuracy_score(y_test, predicted)